### This notebook is for scoring the WML model and to make sure the records have reached the Watson OpenScale

In [ ]:
# Add WML credentials. If using Watson cloud instance, copy paste the WML credentials from there.
WML_CREDENTIALS = {
  "apikey": "xxxxx",
  "iam_apikey_description": "Auto-generated for key xxxxx",
  "iam_apikey_name": "Service credentials-1",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/xxxxx::serviceid:ServiceId-xxxxx",
  "instance_id": "xxxxx",
  "url": "https://us-south.ml.cloud.ibm.com"
}

In [ ]:
# Your user API Key
CLOUD_API_KEY='xxxxx'

In [ ]:
!pip install --upgrade ibm-ai-openscale --no-cache | tail -n 1
!pip install --upgrade watson-machine-learning-client | tail -n 1

In [ ]:
!rm german_credit_data_biased_test_2.csv
!wget https://raw.githubusercontent.com/ravichamarthy/german-credit/master/german_credit_data_biased_test_2.csv

In [ ]:
import pandas as pd
df_data=pd.read_csv ("german_credit_data_biased_test_2.csv")

In [ ]:
len(df_data)

In [ ]:
cols_to_remove = ['Risk']
for col in cols_to_remove:
    if col in df_data.columns:
        del df_data[col] 

fields = df_data.columns.tolist()
values = df_data[fields].values.tolist()

payload_scoring = {"fields": fields,"values": values}

In [ ]:
WML_DEPLOYMENT_ID = 'ae314e81-b1bd-4abc-9260-0a8ad940ce23'
OPENSCALE_SUBSCRIPTION_ID = 'c64afd35-be05-4a4a-9663-170d7cbc25c4'

In [ ]:
def payload_logging():
    from watson_machine_learning_client import WatsonMachineLearningAPIClient    
    wml_client = WatsonMachineLearningAPIClient(WML_CREDENTIALS)
    
    credit_risk_scoring_endpoint = None

    for deployment in wml_client.deployments.get_details()['resources']:
        if WML_DEPLOYMENT_ID in deployment['metadata']['guid']:
            credit_risk_scoring_endpoint = deployment['entity']['scoring_url']

    print(credit_risk_scoring_endpoint)    
    scoring_response = wml_client.deployments.score(credit_risk_scoring_endpoint, payload_scoring)
    
    print('Single record scoring result:', '\n fields:', scoring_response['fields'], '\n values: ', scoring_response['values'][0])
    #print(scoring_response)

In [ ]:
payload_logging()

In [ ]:
from ibm_ai_openscale import APIClient
from ibm_ai_openscale.engines import *
from ibm_ai_openscale.utils import *
from ibm_ai_openscale.supporting_classes import PayloadRecord, Feature
from ibm_ai_openscale.supporting_classes.enums import *

In [ ]:
from ibm_ai_openscale.utils import get_instance_guid


WOS_GUID = get_instance_guid(api_key=CLOUD_API_KEY)
WOS_CREDENTIALS = {
    "instance_guid": WOS_GUID,
    "apikey": CLOUD_API_KEY,
    "url": "https://api.aiopenscale.cloud.ibm.com"
}

if WOS_GUID is None:
    print('Watson OpenScale GUID NOT FOUND')
else:
    print(WOS_GUID)

In [ ]:
ai_client = APIClient(aios_credentials=WOS_CREDENTIALS)
ai_client.version

In [ ]:
data_mart_details = ai_client.data_mart.get_details()
data_mart_details

In [ ]:

subscription = ai_client.data_mart.subscriptions.get(OPENSCALE_SUBSCRIPTION_ID)
subscription

In [ ]:
time.sleep(5)
subscription.payload_logging.get_records_count()